In [12]:
import psycopg2

import pandas as pd
import numpy as np

from copy import deepcopy

import ast
import random
import networkx as nx
import time, unicodedata
import itertools

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from joblib import Parallel, delayed
from datetime import datetime

In [3]:
rc = pd.read_csv("../REST/static/random_connections.csv",index_col="Unnamed: 0")
rc.sample(5)

,from_user_id,to_user_id,formation
12847,715580484,288517297,{'2018.05.11': True}
12987,97707163,833429630,{'2018.05.29': True}
14858,940130879,108026159,{'2018.05.29': True}
1805,828020555,915278292,{'2018.05.26': True}
11560,147572001,110804295,{'2018.05.02': True}


In [14]:
import ast
str2dict = lambda d : ast.literal_eval(d)

In [15]:
rc.formation = rc.formation.apply(str2dict)

In [16]:
def present_in_date(changes_dates, queried_date):
    """
    checking if a connection is present in a queried date
    changes_dates: {d1:True, d2:False, d3:True} connection added or removed
    queried_date: e.g. "2018.05.08"
    """
    str2date = lambda strdate: datetime.strptime(strdate, '%Y.%m.%d')  # 2018.05.08
    changes = sorted(changes_dates,key=lambda d: str2date(d))
    queried_date = datetime.strptime(queried_date, '%Y.%m.%d')
    present = False
    for d in changes:
        if queried_date < str2date(d):
            break
        present = changes_dates[d]
    return present

In [70]:
def calculate_new_edges2(d1="2018.05.01", d2="2018.05.02"):
    nw = deepcopy(rc)
    for_col = nw.formation.apply(lambda dates: present_in_date(dates,d2) and not present_in_date(dates,d1))
    return rc[for_col == True]
    

In [71]:
s = time.time()
c1 = calculate_new_edges()
print("took: ",time.time()-s)

took:  0.3826630115509033


In [72]:
c1.sample(5)

,from_user_id,to_user_id,formation
11593,612625044,334697739,{'2018.05.02': True}
14977,722873773,594065508,{'2018.05.02': True}
7929,87700605,175426030,{'2018.05.02': True}
9866,909735272,314339417,{'2018.05.02': True}
9235,76640116,241277446,{'2018.05.02': True}


In [73]:
def get_connections_by_date(cons, date, present=True):
    nw = deepcopy(cons)
    for_col = nw.formation.apply(lambda dates: present_in_date(dates,date))
    return cons[for_col == present]

In [74]:
def calculate_new_edges(d1="2018.05.01", d2="2018.05.02"):
    return get_connections_by_date(get_connections_by_date(rc, d2), d1, False)

In [118]:
s = time.time()
c2 = calculate_new_edges2()
print("took: ",time.time()-s)

took:  0.35861849784851074


In [119]:
twitter_users = pd.read_csv("../REST/static/twitter_users.csv", index_col="id")
twitter_users.sample(5)

,match_name,screen_name,match_ratio,tw_id,name,lang,followers_count,friends_count,is_org,community
id,,,,,,,,,,
174415744,sehir sehir,sehiredebiyat,95,1.744157e+08,sehir edebiyat,en,2070.0,287.0,True,8
244600920,merve akalin,meurvve,91,2.446009e+09,Merve Çakal,tr,10.0,81.0,False,29
101400688,furkan seza mumcu,furkanseza,95,1.014007e+09,Furkan Mumcu,tr,174.0,208.0,False,25
162081482,ceyda tatari,ceydatatari,100,1.620815e+08,Ceyda Tatari,en,192.0,184.0,False,7
586815260,yonca baran,BaranYonca,100,5.868153e+08,yonca baran,tr,125.0,153.0,False,24


### Categorizing edges (user-user vs user-foci)

In [120]:
orgs = set(twitter_users[twitter_users.is_org==True].index)

In [147]:
is_focal = lambda from_,to, orgs: (0,0) if (from_ in orgs and to in orgs)\
                                        or not(from_ in orgs or to in orgs)\
                                            else ((to,from_) if to in orgs else (from_,to))

In [150]:
c2["foci"] = c2.apply(lambda row:is_focal(row['from_user_id'], row['to_user_id'], orgs), axis=1)
c2["nonfoci"] = c2.foci.apply(lambda f: f[1])
c2["foci"] = c2.foci.apply(lambda f: f[0])
c2[c2.foci!=0]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

,from_user_id,to_user_id,formation,foci,nonfoci
2777,154837447,819269230,{'2018.05.02': True},819269230,154837447
5148,888378156,430605010,{'2018.05.02': True},430605010,888378156
8967,746201372,271718939,{'2018.05.02': True},271718939,746201372
9877,900469680,283216865,{'2018.05.02': True},283216865,900469680
10734,111082356,386276047,{'2018.05.02': True},111082356,386276047
12256,306164644,754963839,{'2018.05.02': True},306164644,754963839
13585,306164644,550121465,{'2018.05.02': True},306164644,550121465
14824,594766612,847741204,{'2018.05.02': True},847741204,594766612


# Calculating membership closures
### if a user u1 just got a new connection with foci f1 at time t2, get friends of u1 who had connections with f1 at time t1

In [152]:
t1 = get_connections_by_date(rc, "2018.05.01")

In [153]:
t1["foci"] = t1.apply(lambda row:is_focal(row['from_user_id'], row['to_user_id'], orgs), axis=1)
t1["nonfoci"] = t1.foci.apply(lambda f: f[1])
t1["foci"] = t1.foci.apply(lambda f: f[0])
t1[t1.foci!=0]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

,from_user_id,to_user_id,formation,foci,nonfoci
697,581066517,347250524,{'2018.05.01': True},581066517,347250524
4193,856911339,74959905,{'2018.05.01': True},856911339,74959905
6091,118400394,271718939,{'2018.05.01': True},271718939,118400394
9466,284399096,131150936,{'2018.05.01': True},131150936,284399096
10415,152042796,819269230,{'2018.05.01': True},819269230,152042796
12308,441584584,803581588,{'2018.05.01': True},803581588,441584584
12366,218228766,111082356,{'2018.05.01': True},111082356,218228766
12708,306672777,841682908,{'2018.05.01': True},841682908,306672777
13396,856911339,946886773,{'2018.05.01': True},856911339,946886773
13581,847741204,110320190,{'2018.05.01': True},847741204,110320190


In [154]:
# foci'es present at t1 and t2
mutual_foci = c2[c2.foci!=0].merge(t1[t1.foci!=0], on="foci")
mutual_foci

,from_user_id_x,to_user_id_x,formation_x,foci,nonfoci_x,from_user_id_y,to_user_id_y,formation_y,nonfoci_y
0,154837447,819269230,{'2018.05.02': True},819269230,154837447,152042796,819269230,{'2018.05.01': True},152042796
1,746201372,271718939,{'2018.05.02': True},271718939,746201372,118400394,271718939,{'2018.05.01': True},118400394
2,111082356,386276047,{'2018.05.02': True},111082356,386276047,218228766,111082356,{'2018.05.01': True},218228766
3,594766612,847741204,{'2018.05.02': True},847741204,594766612,847741204,110320190,{'2018.05.01': True},110320190


## check for membership closure

In [146]:
def membership_closure(cons, u1, u2):
    truncate = lambda x: int(str(int(x))[:9])
    u1_friends = set(
        cons.apply(lambda row: truncate(row['to_user_id'])if truncate(u1) == truncate(row['from_user_id'])\
                 else (truncate(row['from_user_id']) if truncate(u1) == truncate(row['to_user_id'])\
                   else np.NaN), axis=1).dropna())
    return u2 in u1_friends

In [156]:
mutual_foci['membership_closure'] = mutual_foci.apply(lambda row: 
                                                      membership_closure(t1,
                                                                         row['nonfoci_x'],
                                                                         row['nonfoci_y']),axis=1)
mutual_foci

,from_user_id_x,to_user_id_x,formation_x,foci,nonfoci_x,from_user_id_y,to_user_id_y,formation_y,nonfoci_y,membership_closure
0,154837447,819269230,{'2018.05.02': True},819269230,154837447,152042796,819269230,{'2018.05.01': True},152042796,False
1,746201372,271718939,{'2018.05.02': True},271718939,746201372,118400394,271718939,{'2018.05.01': True},118400394,False
2,111082356,386276047,{'2018.05.02': True},111082356,386276047,218228766,111082356,{'2018.05.01': True},218228766,False
3,594766612,847741204,{'2018.05.02': True},847741204,594766612,847741204,110320190,{'2018.05.01': True},110320190,False


In [40]:
def clean(name, min_len=5, junk_replacement=''):
    try:
        cleaned = unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').lower().decode("ascii")
    except TypeError:
        return junk_replacement
    if len(cleaned) < min_len:
        return junk_replacement
    return cleaned

In [41]:
def construct_network(connections, rec_metrics=True):
    G = nx.DiGraph()
    truncate = lambda x: int(str(int(x))[:9])
    for _, row in connections.iterrows():
        from_ = truncate (row["from_user_id"])
        to = truncate (row["to_user_id"])
        if from_ in twitter_users.index and to in twitter_users.index:
            G.add_edge(from_, to)

    augs = ["name", "screen_name", "match_name", "followers_count", "friends_count", "lang"]
    for node in G.nodes():
        user = twitter_users.loc[node]
        for aug in augs:
            if type(user[aug]) == str:
                m = clean(user[aug])
            else:
                m = user[aug]
            G.nodes[node][aug] = m

    if rec_metrics:
        return recalculate_metrics(G, parse=False)
    return G

In [45]:
twitter_users = pd.read_csv("../REST/static/twitter_users.csv", index_col="id")
g = construct_network(cons, False)

In [46]:
g.nodes()

NodeView((99528248, 280162312, 825982985, 214511627, 174575628, 429101069, 492441614, 282013711, 309387281, 878145554, 378589207, 219985599, 166260766, 521494559, 252747814, 951803948, 846204973, 477970480, 263102513, 303712444, 157065267, 546145630, 68714550, 939433993, 742686779, 242561084, 283967550, 379052096, 367968322, 800456772, 861384775, 235114569, 121806922, 377901136, 119926869, 218439766, 800981689, 244600920, 314339417, 163958799, 859103325, 187936863, 492408930, 955502694, 342192231, 985120873, 829767786, 98857070, 895516783, 101400688, 159965297, 422637683, 105865333, 162017398, 167617087, 885989498, 935448699, 833429630, 181272713, 105140362, 298713229, 312324238, 104079504, 885133463, 165159065, 360181915, 923681478, 939057318, 929796263, 237928616, 223486124, 333914287, 239472818, 487846071, 162081482, 514220222, 116871367, 895381707, 952632696, 169021655, 335220952, 257503461, 221274342, 228970729, 194228463, 756224243, 755388661, 490250488, 531509972, 754963839, 267